In [1]:
!pip install pdfplumber sentence-transformers faiss-cpu transformers langchain groq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5

In [3]:
!pip install langchain-groq


In [4]:
from langchain_groq import ChatGroq

In [6]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [8]:
import os
import pdfplumber
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForSequenceClassification
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Set environment variable for GROQ API
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual key


In [9]:
def extract_text_from_pdf(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
    return full_text

def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

pdf_path = "/content/Sayandeep Resume.pdf"  # Replace with your actual file path
raw_text = extract_text_from_pdf(pdf_path)
chunks = chunk_text(raw_text)


In [10]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks)

index = faiss.IndexFlatL2(len(chunk_embeddings[0]))
index.add(np.array(chunk_embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
def retrieve_context(question, k=3):
    question_vec = embedder.encode([question])
    D, I = index.search(np.array(question_vec), k)
    return [chunks[i] for i in I[0]]


In [12]:
template = """
You are an AI assistant helping with document Q&A.
Answer the question using only the information from the context below.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [13]:
llm = ChatGroq(model_name="gemma2-9b-it", temperature=0.2)
qa_chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-13-1641002576.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(llm=llm, prompt=prompt)


In [14]:
def generate_answer_langchain(context_chunks, question):
    context = " ".join(context_chunks)
    return qa_chain.run({"context": context, "question": question})


In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    'vectara/hallucination_evaluation_model', trust_remote_code=True
)

def detect_hallucination(premise, hypothesis):
    score = model.predict([(premise, hypothesis)])
    return score[0]


config.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

configuration_hhem_v2.py:   0%|          | 0.00/760 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vectara/hallucination_evaluation_model:
- configuration_hhem_v2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type HHEMv2Config to instantiate a model of type HHEMv2. This is not supported for all configurations of models and can yield errors.


modeling_hhem_v2.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vectara/hallucination_evaluation_model:
- modeling_hhem_v2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [17]:
# Step 1: Ask a question
question = "What are the skills of sayandeep"

# Step 2: Retrieve context
retrieved_chunks = retrieve_context(question)

# Step 3: Generate answer
generated_answer = generate_answer_langchain(retrieved_chunks, question)
print("Generated Answer:", generated_answer)

# Step 4: Check for hallucination
combined_context = " ".join(retrieved_chunks)
hallucination_score = detect_hallucination(combined_context, generated_answer)

print(f"Hallucination Score: {hallucination_score:.4f}")
if hallucination_score < 0.5:
    print("❗ The answer might be hallucinated.")
else:
    print("✅ The answer is consistent with the retrieved context.")


Generated Answer: Sayandeep's skills include:

* **Programming Languages:** C/C++ , Python, Javascript
* **Tools and Frameworks:** Jupyter, Flask, FastAPI, Docker, Langchain, MLflow, Visual Studio , Git , Github, DVC & Github Actions
* **Other Skills:** Machine Learning, Deep Learning, GenAI, MLOps, SQL, DBMS ,OOPS, Operating system,Data Warehousing, Computer Networks
* **Math Courses:** Industrial Statistics (Descriptive and Inferential), Probability, Linear Algebra, Calculus 


He is also a Codeforces Specialist (1400*), has solved 550+ Data Structures and Algorithms questions on Leetcode, and is a 3 star on Codechef (1702). 

Hallucination Score: 0.8823
✅ The answer is consistent with the retrieved context.
